In [1]:
%pip install pandas
%pip install numpy
import pandas as pd 
import numpy as np 


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
df = pd.read_csv("eaf_temp.csv")
df.head()


,HEATID,DATETIME,TEMP,VALO2_PPM
0,6F0020,2016-01-01 19:15:00,1650.0,1242.0
1,6F0021,2016-01-01 20:05:00,1649.0,1160.0
2,6F0021,2016-01-01 20:05:00,1649.0,1160.0
3,6F0021,2016-01-01 20:03:00,1606.0,1152.0
4,6F0022,2016-01-01 21:03:00,1636.0,947.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85104 entries, 0 to 85103
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   HEATID     85104 non-null  object 
 1   DATETIME   85104 non-null  object 
 2   TEMP       85104 non-null  float64
 3   VALO2_PPM  85104 non-null  float64
dtypes: float64(2), object(2)
memory usage: 2.6+ MB


In [24]:
df["DATETIME"] = pd.to_datetime(df["DATETIME"])                     
df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85104 entries, 0 to 85103
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   DATETIME   85104 non-null  datetime64[ns]
 1   TEMP       85104 non-null  float64       
 2   VALO2_PPM  85104 non-null  float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 1.9 MB


In [25]:
#make plots of the data
X = df
y = df[:10]


In [2]:
df_final = pd.read_csv("df_final.csv")
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356830 entries, 0 to 356829
Data columns (total 23 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       356830 non-null  int64  
 1   POSITIONROW      356830 non-null  float64
 2   DATETIME         356830 non-null  object 
 3   VALC             356830 non-null  float64
 4   VALSI            356830 non-null  float64
 5   VALMN            356830 non-null  float64
 6   VALP             356830 non-null  float64
 7   VALS             356830 non-null  float64
 8   VALCU            356830 non-null  float64
 9   VALCR            356830 non-null  float64
 10  VALMO            356830 non-null  float64
 11  VALNI            356830 non-null  float64
 12  TAP              356830 non-null  float64
 13  STARTTIME        356830 non-null  object 
 14  MW               356830 non-null  float64
 15  TEMP             356830 non-null  float64
 16  VALO2_PPM        356830 non-null  floa

In [ ]:
import torch
from torch import nn
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

# Define the number of input features
n_features = df.shape[1] - 1  # subtract 1 because 'MW' is the target, not an input feature

# Define the network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_features, 128)  # First layer
        self.fc2 = nn.Linear(128, 64)  # Second layer
        self.fc3 = nn.Linear(64, 1)  # Output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
# Define the number of input features
n_features = df_final.shape[1] - 1  # subtract 1 because 'MW' is the target, not an input feature

# Define the network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_features, 128)  # First layer
        self.fc2 = nn.Linear(128, 64)  # Second layer
        self.fc3 = nn.Linear(64, 1)  # Output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print(net)

# Prepare the data
class MyDataset(Dataset):
    def __init__(self, df):
        self.X = df.drop('MW', axis=1).values
        self.y = df[['MW']].values

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

dataset = MyDataset(df_final)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define a loss function and an optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

# Train the network
for epoch in range(100):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.float())
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    print('Epoch %d loss: %.3f' % (epoch + 1, running_loss / len(dataloader)))

print('Finished Training')
print(net)
